In [56]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit
from pyspark.sql.types import IntegerType

spark = SparkSession\
        .builder\
        .appName("ALS").config("spark.driver.host","localhost")\
        .getOrCreate()

In [57]:
# read in the dataset into pyspark DataFrame
attendance = spark.read.csv('./attend.csv', header='true', inferSchema = 'true')
attendance.head(5)

[Row(уникальный номер занятия=401346550, уникальный номер группы=801346550, уникальный номер участника=101352023, направление 2='ОНЛАЙН Гимнастика', направление 3='ОНЛАЙН Цигун', онлайн/офлайн='Да', дата занятия='2022-08-01', время начала занятия='09:00:00', время окончания занятия='10:00:00'),
 Row(уникальный номер занятия=401346550, уникальный номер группы=801346550, уникальный номер участника=101385462, направление 2='ОНЛАЙН Гимнастика', направление 3='ОНЛАЙН Цигун', онлайн/офлайн='Да', дата занятия='2022-08-01', время начала занятия='09:00:00', время окончания занятия='10:00:00'),
 Row(уникальный номер занятия=401346550, уникальный номер группы=801346550, уникальный номер участника=101421897, направление 2='ОНЛАЙН Гимнастика', направление 3='ОНЛАЙН Цигун', онлайн/офлайн='Да', дата занятия='2022-08-01', время начала занятия='09:00:00', время окончания занятия='10:00:00'),
 Row(уникальный номер занятия=401346550, уникальный номер группы=801346550, уникальный номер участника=101354499

In [58]:
attendance.dtypes

[('уникальный номер занятия', 'int'),
 ('уникальный номер группы', 'int'),
 ('уникальный номер участника', 'int'),
 ('направление 2', 'string'),
 ('направление 3', 'string'),
 ('онлайн/офлайн', 'string'),
 ('дата занятия', 'string'),
 ('время начала занятия', 'string'),
 ('время окончания занятия', 'string')]

We aren't going to need the time stamp, so we can go ahead and remove that column.

In [59]:
attendance = attendance.drop('дата занятия').drop('время начала занятия').drop('время окончания занятия').drop('направление 2').drop('направление 3').drop('уникальный номер занятия')
attendance.dtypes

[('уникальный номер группы', 'int'),
 ('уникальный номер участника', 'int'),
 ('онлайн/офлайн', 'string')]

In [60]:
attendance = attendance.withColumn("rating",
                     when((attendance['онлайн/офлайн'] == "Да"), 1).
                     when((attendance['онлайн/офлайн'] == "Нет"), 2)
                     .otherwise(lit("0"))).drop('онлайн/офлайн')

In [64]:
attendance = attendance.withColumn("rating",attendance.rating.cast(IntegerType()))
attendance.head(5)

[Row(уникальный номер группы=801346550, уникальный номер участника=101352023, rating=1),
 Row(уникальный номер группы=801346550, уникальный номер участника=101385462, rating=1),
 Row(уникальный номер группы=801346550, уникальный номер участника=101421897, rating=1),
 Row(уникальный номер группы=801346550, уникальный номер участника=101354499, rating=1),
 Row(уникальный номер группы=801346550, уникальный номер участника=101421312, rating=1)]

### Fitting the Alternating Least Squares Model

Because this dataset is already preprocessed for us, we can go ahead and fit the Alternating Least Squares model.

* Import the ALS module from pyspark.ml.recommendation.
* Use the randomSplit method on the pyspark DataFrame to separate the dataset into a training and test set
* Fit the Alternating Least Squares Model to the training dataset. Make sure to set the userCol, itemCol, and ratingCol to the appropriate names given this dataset. Then fit the data to the training set and assign it to a variable model. 

In [84]:
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS
# split into 
(training, test) = attendance.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5,rank=4, regParam=0.01, userCol="уникальный номер участника", itemCol="уникальный номер группы", ratingCol="rating",
          coldStartStrategy="drop")

# fit the ALS model to the training set
model = als.fit(training)


Now you've fit the model, and it's time to evaluate it to determine just how well it performed.

* import the RegressionEvalutor from pyspark.ml.evaluation
* generate predictions with your model for the test set by using the `transform` method on your ALS model
* evaluate your model and print out the RMSE from your test set

In [85]:
# importing appropriate library
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.32954920535419624


### Cross Validation to Find the Optimal Model

Let's now find the optimal values for the parameters of the ALS model. Use the built-in Cross Validator in pyspark with a suitable param grid and determine the optimal model. Try with the parameters:

* regularization = [0.01,0.001,0.1])
* rank = [4,10,50]



In [68]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


als_model =  ALS(userCol="уникальный номер участника", itemCol="уникальный номер группы", ratingCol="rating", coldStartStrategy="drop")

                 
params = ParamGridBuilder().addGrid(als_model.regParam, [0.01,0.001,0.1]).addGrid(als_model.rank, [4,10,50]).build()


## instantiating crossvalidator estimator
cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(attendance)    

# We see the best model has a rank of 50, so we will use that in our future models with this dataset
best_model.bestModel.rank

23/05/22 23:55:06 WARN BlockManager: Block rdd_247_0 already exists on this machine; not re-adding it
23/05/22 23:55:06 WARN BlockManager: Block rdd_247_0 already exists on this machine; not re-adding it
23/05/22 23:55:06 WARN BlockManager: Block rdd_247_0 already exists on this machine; not re-adding it


50

In [77]:
best_model.bestModel

ALSModel: uid=ALS_807e18f91ad4, rank=50

### Fitting the Alternating Least Squares Model with rank 50

In [86]:
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS
# split into 
(training, test) = attendance.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5,rank=50, regParam=0.01, userCol="уникальный номер участника", itemCol="уникальный номер группы", ratingCol="rating",
          coldStartStrategy="drop")

# fit the ALS model to the training set
model = als.fit(training)

In [87]:
# importing appropriate library
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.06521803776431001


## Getting Recommendations

Now it's time to actually get some recommendations! The ALS model has built in methods called `recommendForUserSubset` and `recommendForAllUsers`.

In the next line, we are creating an RDD with the top 10 recommendations for every user and then selecting one user to find out his predictions:

In [81]:
recommendations = model.recommendForAllUsers(10)
recommendations.where(recommendations["уникальный номер участника"] == 101352023).collect()

[Row(уникальный номер участника=101352023, recommendations=[Row(уникальный номер группы=801368559, rating=2.6092336177825928), Row(уникальный номер группы=801361070, rating=2.6085448265075684), Row(уникальный номер группы=801349882, rating=2.598538398742676), Row(уникальный номер группы=801367536, rating=2.464735984802246), Row(уникальный номер группы=801367541, rating=2.397475004196167), Row(уникальный номер группы=801352724, rating=2.395500659942627), Row(уникальный номер группы=801361139, rating=2.3931448459625244), Row(уникальный номер группы=801371881, rating=2.3717169761657715), Row(уникальный номер группы=801359089, rating=2.3572616577148438), Row(уникальный номер группы=801349262, rating=2.341343402862549)])]

In [38]:
spark.stop()